In [1]:
import gym

env = gym.make("FrozenLake-v0").env

In [ ]:
"""
Documentation from https://gym.openai.com/envs/FrozenLake-v0/
SFFF       (S: starting point, safe)
FHFH       (F: frozen surface, safe)
FFFH       (H: hole, fall to your doom)
HFFG       (G: goal, where the frisbee is located)
"""

In [3]:
# reset ad display environment
env.reset()
env.render()


SFFF
FHFH
FFFH
HFFG


In [4]:
# action space are possible moves: ["Left", "Down", "Right", "Up"] (https://github.com/openai/gym/blob/master/gym/envs/toy_text/frozen_lake.py)
print("Action Space {}".format(env.action_space))
# states are the possible positions : 4x4
print("State Space {}".format(env.observation_space))

Action Space Discrete(4)
State Space Discrete(16)


In [5]:
state, reward, done, info = env.step(1)
env.render()
print(state, reward, done, info)

  (Down)
SFFF
FHFH
FFFH
HFFG
1 0.0 False {'prob': 0.3333333333333333}


In [6]:
# Reward table called P: states x actions
# {action: [(probability, nextstate, reward, done)]}
# probability of taking the action can be lower than one: "the ice is slippery, so you won't always move in the direction you intend."
env.P[4]

{0: [(0.3333333333333333, 0, 0.0, False),
  (0.3333333333333333, 4, 0.0, False),
  (0.3333333333333333, 8, 0.0, False)],
 1: [(0.3333333333333333, 4, 0.0, False),
  (0.3333333333333333, 8, 0.0, False),
  (0.3333333333333333, 5, 0.0, True)],
 2: [(0.3333333333333333, 8, 0.0, False),
  (0.3333333333333333, 5, 0.0, True),
  (0.3333333333333333, 0, 0.0, False)],
 3: [(0.3333333333333333, 5, 0.0, True),
  (0.3333333333333333, 0, 0.0, False),
  (0.3333333333333333, 4, 0.0, False)]}

In [7]:
env.reset()

imgs = []

done = False

while not done:
    action = env.action_space.sample()
    state, reward, done, info = env.step(action)
    
    imgs.append({
        'frame': env.render(mode='ansi'),
        'state': state,
        'action': action,
        'reward': reward
        }
    )

In [8]:
from IPython.display import clear_output
from time import sleep

def print_frames(frames):
    for i, img in enumerate(imgs):
        clear_output(wait=True)
        print(img['frame'])
        print(f"Step: {i + 1}")
        print(f"State: {img['state']}")
        print(f"Action: {img['action']}")
        print(f"Reward: {img['reward']}")
        sleep(.5)
        
print_frames(imgs)

  (Right)
SFFF
FHFH
FFFH
HFFG

Step: 5
State: 5
Action: 2
Reward: 0.0


In [9]:
import numpy as np
q_table = np.zeros([env.observation_space.n, env.action_space.n])

In [10]:
import random

# parameter for q-table
alpha = 0.1
gamma = 0.9
epsilon = 0.3

for i in range(1, 100001):
    state = env.reset()

    done = False    
    while not done:
        if random.uniform(0, 1) < epsilon:
            action = env.action_space.sample()
        else:
            action = np.argmax(q_table[state])

        next_state, reward, done, info = env.step(action)
        # set specific reward to add penalties
        if done and reward != 1:
            reward = -10
        elif reward == 0:
            reward = -0.5
        else:
            reward = 10

        old_value = q_table[state, action]
        next_max = np.max(q_table[next_state])
        
        new_value = (1 - alpha) * old_value + alpha * (reward + gamma * next_max)
        q_table[state, action] = new_value

        state = next_state
        
    if i % 100 == 0:
        clear_output(wait=True)
        print(f"Try n°{i}")


Try n°100000


In [11]:
# qtable for the case at the left of the goal
q_table[14]

array([-0.48412002,  5.12513771,  2.56558272,  0.47564205])

In [16]:
env.step(1)
env.render()

  (Down)
SFFF
FHFH
FFFH
HFFG


In [13]:
env.reset()

imgs = []

done = False

while not done:
    action = np.argmax(q_table[state])
    state, reward, done, info = env.step(action)
    
    imgs.append({
        'frame': env.render(mode='ansi'),
        'state': state,
        'action': action,
        'reward': reward
        }
    )

In [25]:
from IPython.display import clear_output
from time import sleep

def print_frames(imgs):
    for i, img in enumerate(imgs):
        clear_output(wait=True)
        print(img['frame'])
        print(f"Step: {i + 1}")
        print(f"State: {img['state']}")
        print(f"Action: {img['action']}")
        print(f"Reward: {img['reward']}")
        sleep(.1)
        
print_frames(imgs)

  (Down)
SFFF
FHFH
FFFH
HFFG

Step: 18
State: 15
Action: 1
Reward: 1.0
